In [7]:
!pip install --upgrade google-auth   google-api-python-client

     |████████████████████████████████| 146 kB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 12.5 MB/s eta 0:00:01
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.30.0
    Uninstalling google-auth-1.30.0:
      Successfully uninstalled google-auth-1.30.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.3.0
    Uninstalling google-api-python-client-2.3.0:
      Successfully uninstalled google-api-python-client-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
witwidget 1.7.0 requires oauth2client>=4.1.3, but you have oauth2client 3.0.0 which is incompatible.
tfx 0.21.5 requires docker<5,>=4.1, but you have docker 5.0.0 which is incompatible.
tfx 0.21.5 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.6.0 whi

In [4]:
print('\nPROJECT_ID=$(gcloud config list project --format "value(core.project)")\nBUCKET=$PROJECT_ID\nREGION="us-central1"\nTFVERSION="2.1"\n\n# Output directory and jobID\nOUTDIR=gs://${BUCKET}/taxifare/trained_model_$(date -u +%y%m%d_%H%M%S)\nJOBID=taxifare_$(date -u +%y%m%d_%H%M%S)\necho ${OUTDIR} ${REGION} ${JOBID}\ngsutil -m rm -rf ${OUTDIR}\n\n# Model and training hyperparameters\nBATCH_SIZE=15\nNUM_EXAMPLES_TO_TRAIN_ON=100\nNUM_EVALS=10\nNBUCKETS=10\nLR=0.001\nNNSIZE="32 8"\n\n# GCS paths\nGCS_PROJECT_PATH=gs://$BUCKET/taxifare\nDATA_PATH=$GCS_PROJECT_PATH/data\nTRAIN_DATA_PATH=$DATA_PATH/taxi-train*\nEVAL_DATA_PATH=$DATA_PATH/taxi-valid*\n\n# TODO\ngcloud ai-platform jobs submit training $JOBID \\\n    # TODO: Your code goes here\n    -- \\\n    --eval_data_path $EVAL_DATA_PATH \\\n    --output_dir $OUTDIR \\\n    --train_data_path $TRAIN_DATA_PATH \\\n    --batch_size $BATCH_SIZE \\\n    --num_examples_to_train_on $NUM_EXAMPLES_TO_TRAIN_ON \\\n    --num_evals $NUM_EVALS \\\n    --nbuckets $NBUCKETS \\\n    --lr $LR \\\n    --nnsize $NNSIZE \n')


PROJECT_ID=$(gcloud config list project --format "value(core.project)")
BUCKET=$PROJECT_ID
REGION="us-central1"
TFVERSION="2.1"

# Output directory and jobID
OUTDIR=gs://${BUCKET}/taxifare/trained_model_$(date -u +%y%m%d_%H%M%S)
JOBID=taxifare_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBID}
gsutil -m rm -rf ${OUTDIR}

# Model and training hyperparameters
BATCH_SIZE=15
NUM_EXAMPLES_TO_TRAIN_ON=100
NUM_EVALS=10
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# GCS paths
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

# TODO
gcloud ai-platform jobs submit training $JOBID \
    # TODO: Your code goes here
    -- \
    --eval_data_path $EVAL_DATA_PATH \
    --output_dir $OUTDIR \
    --train_data_path $TRAIN_DATA_PATH \
    --batch_size $BATCH_SIZE \
    --num_examples_to_train_on $NUM_EXAMPLES_TO_TRAIN_ON \
    --num_evals $NUM_EVALS \
    --nbuckets $NBUCKETS \
    --lr $LR \
    --nnsiz

In [8]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

In [9]:
credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials)
request_data = [
 {'pickup_longitude': -73.885262,
 'pickup_latitude': 40.773008,
 'dropoff_longitude': -73.987232,
 'dropoff_latitude': 40.732403}]
parent = 'projects/%s/models/%s/versions/%s' % ('cloud-training-demos', 'taxifare', 'v1')
response = api.projects().predict(body={'instances': request_data}, 
name=parent).execute()

HttpError: <HttpError 403 when requesting https://ml.googleapis.com/v1/projects/cloud-training-demos/models/taxifare/versions/v1:predict?alt=json returned "Access to model denied.". Details: "Access to model denied.">